In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
%matplotlib inline

In [2]:
# Load dataset
df = pd.read_csv("mtcars.csv")
df.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [3]:
df = df.drop(columns=['model'], errors='ignore')
X = df.drop(columns=['mpg'])
y = df['mpg']

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
mean_squared_error(y_test, y_pred)

9.93574409958809

In [5]:
# Create a DataFrame with features and coefficients
importance_df = pd.DataFrame({
    'Feature': x_train.columns,
    'Coefficient': model.coef_
}).sort_values(by='Coefficient', key=abs, ascending=False)
importance_df

,Feature,Coefficient
4,wt,-3.830001
7,am,1.579418
5,qsec,0.708983
8,gear,0.659154
3,drat,0.598367
0,cyl,-0.401108
9,carb,0.077837
6,vs,0.071637
2,hp,-0.021676
1,disp,0.013166


In [6]:
# Select features
col_imp = ["wt", "am", "qsec", "gear", "drat", "cyl"]
X = df[col_imp]
y = df["mpg"]

# Train-test split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Train linear regression
model = LinearRegression()
model.fit(x_train, y_train)

# Predict
y_pred = model.predict(x_test)
mean_squared_error(y_test, y_pred)

11.664459548183498

Keeping only few important columns increase the MSE score

### Function who will be use in the flask server

In [7]:
def predict(dict_values,col_imp=col_imp, clf=model):
    x = np.array([float(dict_values[col]) for col in col_imp])
    x = x.reshape(1,-1)
    y_pred = clf.predict(x)[0]
    return y_pred

In [8]:
x_test[col_imp]

,wt,am,qsec,gear,drat,cyl
29,2.770,1,15.50,5,3.62,6
15,5.424,0,17.82,3,3.00,8
24,3.845,0,17.05,3,3.08,8
17,2.200,1,19.47,4,4.08,4


In [9]:
x_test[col_imp].iloc[3].T.to_json()

'{"wt":2.2,"am":1.0,"qsec":19.47,"gear":4.0,"drat":4.08,"cyl":4.0}'

In [10]:
x_test_json = {"wt":2.2,"am":1.0,"qsec":19.47,"gear":4.0,"drat":4.08,"cyl":4.0}

In [11]:
predict(x_test_json)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


27.018040590651804

In [12]:
x_test_json2 = {"wt":3.73,"am":1,"qsec":17.6,"gear":4,"drat":3.07,"cyl":8}

In [13]:
predict(x_test_json2)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


17.540976308360932

I save an example just to try the example 

In [171]:
x_test[col_imp].iloc[3].T.to_json("../to_predict_json.json")